## 3. TF-IDF 행렬 및 TDM 행렬 메모리 문제 발견
- 16G 메모리 환경에서 TF-IDF, TDM을 sklearn에서 기본 제공하는 세팅으로는 구현할 수 없어서 필요한 조치들을 담은 노트입니다
- 문서의 수, 최소 단어 등의 옵션을 조절하여 최종 feature셋을 만들고 모델을 시험적으로 구동하였습니다.
    - 위의 시도들이 모두 기각되어서 보다 메모리 용량이 적은 LDA weights를 가지고 feature를 구성하도록 계획했습니다. 이는 다음 노트에서 이어집니다.

In [2]:
#tf-idf 행렬을 사용하지 않고 DTM만을 이용해서 모델 학습을 시켜보기

#1. DTM이용 모델 -> 기각
import pandas as pd
import numpy as np

df = pd.read_pickle('df_noun_2018_re.pkl')

In [5]:
df['cont_noun_str'] = df['cont_noun'].apply(lambda x: ','.join(x))

In [64]:
df.to_pickle('df_noun_2018_re2.pkl')

In [6]:
df.head()

,category,title,mod_content,num_agree,new_status,mod_begin,cont_noun,cont_noun_str
0,행정,아...이런데서 이상한 글이 올라오네요...,"국민청원하는 게시판에서 뭐 섹스,희주야이런 이상한 글 올라오는데 여기는 청원하는데지...",6,0,2018-01-01,"[국민, 청원, 게시판, 섹스, 희, 주야, 글, 청원, 곳, 놈, 처벌, 글]","국민,청원,게시판,섹스,희,주야,글,청원,곳,놈,처벌,글"
1,기타,"현재 사람들 사이에서 유행하는 게임 ""배틀 그라운드"" 에 대한 제재를 가하려 합니다.",배틀 그라운드는 현재 세계적으로나 우리나라 에서나 매우 흥행 하는 게임입니다. 게임...,1,0,2018-01-01,"[배틀, 그라운드, 세계, 나라, 흥행, 게임, 게임, 청소년, 이용, 불가, 게임...","배틀,그라운드,세계,나라,흥행,게임,게임,청소년,이용,불가,게임,표,피시방,청소년,..."
2,정치개혁,양심수 석방 국가보안법 철폐,적폐청산의 시금석 양심수들의 특별사면 배제 문재인 정부를 규탄한다. 지난 문재인 정...,0,0,2018-01-01,"[적폐, 청산, 시금석, 양심수, 특별, 사면, 배제, 문재, 정부, 규탄, 문재,...","적폐,청산,시금석,양심수,특별,사면,배제,문재,정부,규탄,문재,정부,특별,사면,단행..."
3,인권/성평등,민주화유공자들에 대한 국가유공자 인정 요청,"안녕하세요. 살벌한 독재시대를 떠나, 대한민국 민주화를 이룩하신 수많은 열사들이, ...",3,0,2018-01-01,"[안녕, 독재, 시대, 대한민국, 민주, 열사, 민주, 유공자, 인정, 국가, 유공...","안녕,독재,시대,대한민국,민주,열사,민주,유공자,인정,국가,유공자,인정,이야기,청원..."
4,외교/통일/국방,UAE 관련,Uae 괸련한 야당 질의와 관련하여 기본적으로 유시민 작가의 말씀에 동의하여 국익에...,0,0,2018-01-01,"[련, 야당, 질의, 관련, 기본, 유시민, 작가, 말씀, 동의, 국익, 관련, 정...","련,야당,질의,관련,기본,유시민,작가,말씀,동의,국익,관련,정부,문재,정부,참고,야..."


In [7]:
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
cv = CountVectorizer(max_features = 2000, max_df = 0.80, min_df = 5).fit(df['cont_noun_str'])

In [9]:
#dtm 생성
cv_array = cv.transform(df['cont_noun_str']).toarray()

In [11]:
cv_array.shape

(269242, 2000)

In [12]:
#랜덤 포레스트 이용
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [13]:
X = cv_array
y = df.category

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3,
                                                   stratify = y)

In [15]:
rt_clf = RandomForestClassifier(n_jobs = -1, random_state = 156)
rt_clf.fit(X_train, y_train)

RandomForestClassifier(n_jobs=-1, random_state=156)

In [16]:
pred = rt_clf.predict(X_test)

In [17]:
print('DTM이용 랜덤 포레스트 정확도:', accuracy_score(y_test, pred))
    #모델 정확도의 변화가 없음

DTM이용 랜덤 포레스트 정확도: 0.49927574808413705


In [19]:
#나이브 베이즈 분류기 이용
from sklearn.naive_bayes import MultinomialNB

nb_clf = MultinomialNB()
nb_clf.fit(X_train, y_train)

MultinomialNB()

In [21]:
pred = nb_clf.predict(X_test)
print('DTM이용 나이브 베이즈 정확도:', accuracy_score(y_test, pred))
    #어제와 비교하여 낮지만 비슷한 수준

DTM이용 나이브 베이즈 정확도: 0.48659824446287747


In [22]:
#2. 단어의 수를 늘리자 (2000 -> 5000) -> 기각
from sklearn.feature_extraction.text import TfidfVectorizer

tfidfv = TfidfVectorizer(max_features = 5000, max_df = 0.80, min_df = 5,
                        sublinear_tf = True).fit(df['cont_noun_str'])

In [23]:
ti_array = tfidfv.transform(df['cont_noun_str']).toarray()

In [25]:
ti_array.shape

(269242, 5000)

In [26]:
#학습시작
X_train, X_test, y_train, y_test = train_test_split(ti_array, df.category, test_size = 0.3,
                                                   stratify = y)

In [29]:
rt_clf = RandomForestClassifier(n_jobs = -1, random_state = 156)
rt_clf.fit(X_train, y_train)

RandomForestClassifier(n_jobs=-1, random_state=156)

In [30]:
pred = rt_clf.predict(X_test)
print('5000개 단어 이용 랜덤 포레스트 정확도:', accuracy_score(y_test, pred))

5000개 단어 이용 랜덤 포레스트 정확도: 0.5101457170093967


In [32]:
#단어개수를 지정하지 않고 
#최소 문서 등장 개수와 최대 등장 문서 한계선을 저장하면 몇개의 단어가 추출될까?

cv = CountVectorizer(max_df = 0.80, min_df = 5).fit(df['cont_noun_str'])
cv_array = cv.transform(df['cont_noun_str']).toarray()
    #41890개의 단어들로 추출됨
    #메모리 부족 문제로 array 생성 불가

MemoryError: Unable to allocate array with shape (269242, 41890) and data type int64

In [40]:
#3. 문서의 개수를 줄이자 -> 기각
#임의로 한달을 설정하여서 추출된 데이터의 모델 학습

mask = (df.mod_begin >= '2018-08-01') & (df.mod_begin < '2018-09-01')
df2 = df.loc[mask]
df2.reset_index(drop = True, inplace = True)

In [41]:
df2.info()
    #25161개의 문서 존재

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25161 entries, 0 to 25160
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   category       25161 non-null  object        
 1   title          25161 non-null  object        
 2   mod_content    25161 non-null  object        
 3   num_agree      25161 non-null  int64         
 4   new_status     25161 non-null  int64         
 5   mod_begin      25161 non-null  datetime64[ns]
 6   cont_noun      25161 non-null  object        
 7   cont_noun_str  25161 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 1.5+ MB


In [43]:
tfidfv = TfidfVectorizer(max_df = 0.80, min_df = 5,
                        sublinear_tf = True).fit_transform(df2['cont_noun_str'])
tf_array = tfidfv.toarray()

In [44]:
tf_array.shape

(25161, 13377)

In [46]:
X_train, X_test, y_train, y_test = train_test_split(tf_array , df2.category, test_size = 0.3,
                                                   stratify = df2.category)

In [47]:
rt_clf = RandomForestClassifier(n_jobs = -1, random_state = 156)
rt_clf.fit(X_train, y_train)

RandomForestClassifier(n_jobs=-1, random_state=156)

In [49]:
pred = rt_clf.predict(X_test)
print('8월 문서 이용 랜덤 포레스트 정확도:', accuracy_score(y_test, pred))
    #정확도 향상에 영향을 주지 않음. 기각

8월 문서 이용 랜덤 포레스트 정확도: 0.49013114319777457


In [35]:
#LDA이용 차원축소?
#https://towardsdatascience.com/dimensionality-reduction-with-latent-dirichlet-allocation-8d73c586738c
#왜 LDA를 이용해야할까
#LDA를 통해 생성된 문서별 topic에 속할 weight를 바탕으로
#머신러닝을 학습하여 보다 정확한 클래스 분류를 하기 위함
#자연스럽게 학습 데이터의 column은 topic의 개수로 귀결되므로 효율적인 학습이 가능할것

#카테고리 다시 확인
df.category.value_counts()
    #총 17개의 카테고리
    #먼저 n = 17로 설정하여 주제들의 분포를 확인해보자

정치개혁           39425
기타             32134
교통/건축/국토       21747
인권/성평등         21628
외교/통일/국방       20150
안전/환경          19816
일자리            16840
보건복지           16209
육아/교육          16040
행정             14271
문화/예술/체육/언론    13810
경제민주화          12866
미래             12476
성장동력            5431
반려동물            2689
저출산/고령화대책       2348
농산어촌            1362
Name: category, dtype: int64

In [50]:
from gensim import corpora
gen_dict = corpora.Dictionary(df['cont_noun'])

In [67]:
type(gen_dict)

gensim.corpora.dictionary.Dictionary

In [51]:
len(gen_dict)
    #98878개의 단어가 학습됨

98878

In [52]:
gen_dict.token2id

{'게시판': 0,
 '곳': 1,
 '국민': 2,
 '글': 3,
 '놈': 4,
 '섹스': 5,
 '주야': 6,
 '처벌': 7,
 '청원': 8,
 '희': 9,
 '간청': 10,
 '게임': 11,
 '경찰관': 12,
 '관심': 13,
 '권리': 14,
 '그라운드': 15,
 '나라': 16,
 '나이': 17,
 '배틀': 18,
 '보호': 19,
 '불가': 20,
 '설정': 21,
 '세계': 22,
 '신고': 23,
 '오버': 24,
 '워치': 25,
 '이용': 26,
 '제재': 27,
 '제한': 28,
 '중독': 29,
 '지난번': 30,
 '청소년': 31,
 '출동': 32,
 '침해': 33,
 '폭력': 34,
 '표': 35,
 '플레이': 36,
 '피시방': 37,
 '현상': 38,
 '효과': 39,
 '흥행': 40,
 '공작': 41,
 '광주': 42,
 '국가': 43,
 '권력': 44,
 '규명': 45,
 '규탄': 46,
 '내란': 47,
 '노총': 48,
 '누명': 49,
 '눈치': 50,
 '단행': 51,
 '달성': 52,
 '독재': 53,
 '문재': 54,
 '문재인': 55,
 '민생': 56,
 '민주': 57,
 '민중': 58,
 '박근혜': 59,
 '반대': 60,
 '발전소': 61,
 '배신행위': 62,
 '배제': 63,
 '배치': 64,
 '보안법': 65,
 '보장': 66,
 '불국': 67,
 '사드': 68,
 '사면': 69,
 '사상': 70,
 '생활': 71,
 '석방': 72,
 '세력': 73,
 '세월호': 74,
 '소장': 75,
 '수감': 76,
 '수단': 77,
 '시금석': 78,
 '시민': 79,
 '시작': 80,
 '악법': 81,
 '양산': 82,
 '양심수': 83,
 '억압': 84,
 '외면': 85,
 '요구': 86,
 '위원장': 87,
 '유린': 88,
 '유지': 89,
 '음모': 

In [53]:
type(gen_dict.token2id)
    #굳이 gensim의 dictionary를 이용하지 않고
    #sklearn의 CountVectorizer의 vocabulary_를 이용하면 되지 않을까?
    #왜냐하면 sklearn의 CountVectorizer에서 min_df, max_df, max_features를 사용할 수 있기 때문에

dict

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 2000, max_df = 0.80, min_df = 5).fit(df['cont_noun_str'])

cv_dict = cv.vocabulary_
    #gensim의 dictionary와 똑같은 모양이 산출된다

In [ ]:
#변환순서
#1. CountVectorizer로 DTM생성 (CountVectorizer를 생성하는 이유는 
#DTM 생성시 제약조건을 줄 수 있기 때문에 )
#2. CountVectorizer에서 vocabulary_로 단어 정수인덱싱 사전 추출, 
#정수 인덱스 낮은순으로 재정렬
#3. DTM을 scipy의 sparse 메서드 이용하여 csr_matrix로 변환
#4. gensim.matutils.Sparse2Corpus 이용하여 gensim이 이해할 수 있는 corpus로 생성
#5. corpora.Dictionary.from_corpus 메서드 이용 gensim이 이해할 수 있는 dict생성

In [69]:
cv_array = cv.transform(df['cont_noun_str']).toarray()

In [62]:
corpus = [gen_dict.doc2bow(text) for text in df['cont_noun']]
    #cv_dict는 gensim에서 요구하는 attribute가 존재하지 않아서 학습이 되지 않음

In [71]:
#csr_matrix로 변환작업
#sklearn CountVectorizer로 생성된 단어들을 
from scipy import sparse
s_cv_array = sparse.csr_matrix(cv_array)

In [72]:
corpus2 = gensim.matutils.Sparse2Corpus(s_cv_array, documents_columns=False)

In [83]:
cv_dict_sorted = dict(sorted(cv_dict.items(), key = lambda x: x[1]))

In [84]:
type(cv_dict_sorted)

dict

In [88]:
dict(enumerate(cv_dict_sorted))

{0: '가게',
 1: '가격',
 2: '가계',
 3: '가구',
 4: '가난',
 5: '가능',
 6: '가량',
 7: '가상',
 8: '가스',
 9: '가슴',
 10: '가요',
 11: '가운데',
 12: '가입',
 13: '가입자',
 14: '가장',
 15: '가정',
 16: '가족',
 17: '가중',
 18: '가짜',
 19: '가치',
 20: '가해자',
 21: '각하',
 22: '간부',
 23: '간호사',
 24: '갈등',
 25: '감당',
 26: '감독',
 27: '감면',
 28: '감사',
 29: '감소',
 30: '감수',
 31: '감시',
 32: '감안',
 33: '감옥',
 34: '감정',
 35: '감형',
 36: '강간',
 37: '강남',
 38: '강도',
 39: '강력',
 40: '강사',
 41: '강아지',
 42: '강요',
 43: '강원랜드',
 44: '강제',
 45: '강조',
 46: '강화',
 47: '개개인',
 48: '개념',
 49: '개미',
 50: '개발',
 51: '개선',
 52: '개설',
 53: '개월',
 54: '개인',
 55: '개입',
 56: '개정',
 57: '개최',
 58: '개편',
 59: '개헌',
 60: '개혁',
 61: '거기',
 62: '거대',
 63: '거래',
 64: '거래소',
 65: '거리',
 66: '거부',
 67: '거절',
 68: '거주',
 69: '거짓',
 70: '거짓말',
 71: '거품',
 72: '걱정',
 73: '건가요',
 74: '건강',
 75: '건데',
 76: '건물',
 77: '건설',
 78: '건설사',
 79: '건의',
 80: '건지',
 81: '건축',
 82: '걸까요',
 83: '걸로',
 84: '검사',
 85: '검색',
 86: '검증',
 87: '검찰',
 88: '검찰청',
 89: '검토',
 90: '

In [89]:
sk_dict = corpora.Dictionary.from_corpus(corpus2,
                                        id2word = dict(enumerate(cv_dict_sorted)))

In [90]:
NUM_topics = 17
ldamodel = gensim.models.ldamodel.LdaModel(corpus2, 
                                           num_topics = NUM_topics, 
                                           id2word=sk_dict, passes=15)
    #학습시간 30분 넘은것같음?

In [98]:
#모델 저장
import pickle
with open('ldamodel.pkl', 'wb') as f:
    pickle.dump(ldamodel, f)

In [93]:
#토픽모델링 시각화
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus2, sk_dict)
pyLDAvis.display(vis)
    #총 6개의 그룹의 토픽이 겹쳤고 그 중 16, 17의 경우 매우 유사한 토픽
    #2,4 는 정치개혁과 외교와의 관계를 나타내는 것
    #
    #11개로 카테고리의 개수를 요약할 수 있을듯?
    #카테고리 개수를 5 -> 8 -> 11 형태로 지정해서 토픽모델링 결과에 따른 분류 결과를 봐보기
    #람다값은 사용자가 조정가능한데 1에 가까울수록 토픽내에서 등장하는빈도를 우선순위로 고려하여 단어의 산출순서를조정하고
    #0에 가까울수록 토픽내 빈도는적어도 다른 토픽과 차별되는 단어들을 중심으로 보여줌

In [108]:
#시각화 자료 저장
pyLDAvis.save_html(vis,'lda_vis.html')

In [99]:
X_train, X_test, y_train, y_test = train_test_split(cv_array , df.category, test_size = 0.3,
                                                   stratify = df.category)

In [100]:
#11개의 카테고리로 시도해보고 가중치값 산출해보기

from sklearn.decomposition import LatentDirichletAllocation

topic_num = 11

sk_lda = LatentDirichletAllocation(n_components = topic_num)
sk_lda_f = sk_lda.fit(X_train)
    #이것도 소요시간 30분이 넘은듯
    #다음에 시도할 때는 time으로 시간 측정하기

In [101]:
lda_weights = sk_lda_f.transform(X_train)

In [103]:
lda_weights.shape

(188469, 11)

In [104]:
from xgboost import XGBClassifier

xgbc = XGBClassifier(n_estimators = 200, n_jobs = -1)

xgbc.fit(lda_weights, y_train)

XGBClassifier(n_estimators=200, n_jobs=-1, objective='multi:softprob')

In [106]:
#참고한 자료에서 테스트 데이터를 무엇으로 사용했는지에 대한 정보가 없음
#그렇다면 학습 데이터를 가지고 평가?
pred = xgbc.predict(lda_weights)
accuracy_score(y_train, pred)

0.3773617942473298